In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [4]:
# Filter dataframe where total vote count is greater than or equal to 20
vine_review_df = df.filter("total_votes>=20")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37480006|R1B4753QMTD8ON|B00U6AE4BK|     632231911|Maxpedition Chubb...|          Sports|          5|           32|         32|   N|                N|Maxpedition Organ...|As a preamble, ha...| 2015-08-31|
|         US|   49868087|R28BVHOEL6GCHZ|B011CCGRX8|     403987913|5 Resistance Loop...|          Sports|          5|    

In [6]:
# Filter the dataframe to retrieve all the rows where the number of helpful_votes divided by total_votes is equal or greater than 50%
vine_review_fifty_percent_df = vine_review_df.filter(vine_review_df["helpful_votes"]/vine_review_df["total_votes"]>=0.5)
vine_review_fifty_percent_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37480006|R1B4753QMTD8ON|B00U6AE4BK|     632231911|Maxpedition Chubb...|          Sports|          5|           32|         32|   N|                N|Maxpedition Organ...|As a preamble, ha...| 2015-08-31|
|         US|   49868087|R28BVHOEL6GCHZ|B011CCGRX8|     403987913|5 Resistance Loop...|          Sports|          5|    

In [7]:
# Filter the dataframe to retrieve all rows where a review was written as part of the Vine program (paid)
vine_program_df = vine_review_fifty_percent_df.filter(vine_review_fifty_percent_df['vine']=='Y')
vine_program_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51238346|R32OMS1LSDRCOB|B0118XMTV0|     454318751|Slendertone Abs A...|          Sports|          4|           27|         31|   Y|                N|Fun and Potential...|I have heard some...| 2015-08-31|
|         US|   31282801|R2IPKK4I0XRCFR|B00R3MZU80|     470964994|AmazonBasics Resi...|          Sports|          4|    

In [8]:
# Filter the dataframe to retrieve all rows where a review was written not as part of the Vine program (unpaid)
non_vine_program_df = vine_review_fifty_percent_df.filter(vine_review_fifty_percent_df['vine']=='N')
non_vine_program_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37480006|R1B4753QMTD8ON|B00U6AE4BK|     632231911|Maxpedition Chubb...|          Sports|          5|           32|         32|   N|                N|Maxpedition Organ...|As a preamble, ha...| 2015-08-31|
|         US|   49868087|R28BVHOEL6GCHZ|B011CCGRX8|     403987913|5 Resistance Loop...|          Sports|          5|    

In [15]:
# Total Vine program reviews
vine_program_count = vine_program_df.count()
vine_program_count

334

In [16]:
# Total non-Vine program reviews
non_vine_program_count = non_vine_program_df.count()
non_vine_program_count

61614

In [22]:
# Vine program 5 star reviews
vine_five_star_reviews = vine_program_df.filter(vine_program_df['star_rating']== 5).count()
vine_five_star_reviews

139

In [23]:
# Non-Vine program 5 star reviews
non_vine_five_star_reviews = non_vine_program_df.filter(non_vine_program_df['star_rating']== 5).count()
non_vine_five_star_reviews

32665

In [24]:
# Vine program 5 star review percentage
vine_five_star_percentage = (vine_five_star_reviews / vine_program_count)*100
vine_five_star_percentage

41.61676646706587

In [26]:
# Non-Vine program 5 star review percentage
non_vine_five_star_percentage = (non_vine_five_star_reviews / non_vine_program_count)*100
non_vine_five_star_percentage

53.01554841432142